# 영업 성공 여부 분류 경진대회

## 1. 사전 라이브러리 준비 및 데이터 확인
> 라이브러리 download 및 import <br>
> 사전 SEED 고정

In [140]:
!pip install pandas numpy scikit-learn nltk catboost
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
%matplotlib inline

import warnings
warnings.filterwarnings(action="ignore")
nltk.download('punkt')
pd.set_option('display.max_rows', None)
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wlsyo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wlsyo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\wlsyo\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [141]:
# 시드 고정
import os

SEED=42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)

## 2. 데이터 로드 및 확인
> 데이터 로드 후 id, 타겟 컬럼 삭제 <br>
> 간단하게 수치형 컬럼과 범주형 컬럼 확인

In [142]:
train = pd.read_csv("train.csv") # 학습용 데이터
test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [143]:
# 학습 타겟 데이터
targets=train['is_converted']
rows=train.shape[0]

# 
train=train.drop('is_converted',axis=1)
test=test.drop('id',axis=1)
test=test.drop('is_converted',axis=1)

In [144]:
# columns
cols_by_type={}

cols_by_type['categorical']=train.columns[train.dtypes=='object'].tolist()
cols_by_type['numerical']=train.columns[train.dtypes!='object'].tolist()
#colsByType['numerical'].remove('is_converted')

print('\nnumerical columns: '+str(len(cols_by_type['numerical'])))
print('categorical columns: '+str(len(cols_by_type['categorical'])))
print('total columns: '+str(len(cols_by_type['numerical'])+len(cols_by_type['categorical'])))


numerical columns: 13
categorical columns: 15
total columns: 28


## 2. 데이터 전처리
> 범주형은 permitted 딕셔너리에 담긴 허용범위의 변수들로 새로 범주화 진행 <br>
> 수치형은 Robust scaling 진행
> 각 컬럼 별 처리 내용 참조

### 2.1. 허용 변수인 permitted 딕셔너리
> customer_job, product_category, customer_position, inquiry_type, customer_type, expected_timeline : 표제어 추출 혹은 단어 기준으로 나누어 Train 데이터셋 기준 빈도수가 놓은 상위 (표제어 or 단어)로 범주화 <br><br>
> customer_country : / 기준으로 단어 분할 진행 후 빈도수 상위 100개의 단어를 휴먼체크하여 사용, 추가로 gpt로 출현 단어 전체 중 나라이름 색출 후 병합하여 사용 <br><br>
> region : 새로운 칼럼으로 response_corporate 컬럼에서 각 지사별 위치를 기준으로 나눔, 분류 기준은 링크 참조 <br>
> https://www.lge.co.kr/company/info/overseas <br>
> http://www3.lge.co.kr/ir/management_info/organization/ir02.shtml

In [145]:
permitted={
    "customer_job":{"engin":"EN", "administr":"AD", "educ":"ED", "sale":"SA", "inform":"IF", "oper":"OP", "purchas":"PA", "art":"AT", "busi":"BS", "manag":"MA", "dump_key":"OT", "technolog":"TC", "develop":"DV", "consult":"CS", "media":"ME"},
    
    "product_category":{"signag":"SG", "board":"BD", "multi-split":"MS", "single-split":"SS", "tv":"TV", "vrf":"VR", "dump_key":["DB","VR","SG","SS","MS","TV","IT","DT","VI","IT","DT","VI","WL","LD","SE"], "interact":"IT", "digit":"DT","video":"VI","wall":"WL","led":"LD","seri":"SE"},
    
    "customer_country":{
        "india": 1, "brazil": 1, "unitedstates": 1, "OT": 1, "mexico": 1, "philippines": 1, "colombia": 1, "u.a.e": 1, "unitedkingdom": 1, "saudiarabia": 1, "chile": 1, "italy": 1, "peru": 1, "germany": 1, "poland": 1, "egypt": 1,
        "vietnam": 1, "spain": 1, "argentina": 1, "hongkong": 1, "australia": 1, "panama": 1, "france": 1, "canada": 1, "turkey": 1, "ecuador": 1, "indonesia": 1, "t\u00fcrkiye": 1, "singapore": 1, "southafrica": 1, "iraq": 1,
        "nigeria": 1, "thailand": 1, "hungary": 1, "portugal": 1, "kenya": 1, "malaysia": 1, "bulgaria": 1, "costarica": 1, "dominicanrepublic": 1, "israel": 1, "oman": 1, "elsalvador": 1, "pakistan": 1, "guatemala": 1,
        "kuwait": 1, "bangladesh": 1, "qatar": 1, "switzerland": 1, "china": 1, "bolivia": 1, "honduras": 1, "lebanon": 1, "taiwan": 1, "netherlands": 1, "belgium": 1, "bahrain": 1, "venezuela": 1, "puertorico": 1, "greece": 1,
        "japan": 1, "afghanistan": 1, "algeria": 1, "morocco": 1, "romania": 1, "ghana": 1, "jordan": 1, "croatia": 1, "nicaragua": 1, "ireland": 1, "maldives": 1, "serbia": 1, "srilanka": 1, "uruguay": 1, "albania": 1, "jamaica": 1,
        "southkorea": 1, "sweden": 1, "anguilla": 1, "paraguay": 1, "malta": 1, "azerbaijan": 1, "russia": 1, "cambodia": 1, "mozambique": 1, "yemen": 1, "bosniaandherzegovina": 1, "zimbabwe": 1, "iran": 1, "slovenia": 1,
        "ethiopia": 1, "botswana": 1, "papuanewguinea": 1, "senegal": 1, "denmark": 1, "angola": 1, "uganda": 1, "barbados": 1, "laos": 1, "burkinafaso": 1, "congo": 1, "unitedarabemirates": 1, "gambia": 1, "myanmar": 1,
        "togo": 1, "suriname": 1, "mauritius": 1, "czechrepublic": 1, "montenegro": 1, "cameroon": 1, "sierraleone": 1, "ivorycoast": 1, "namibia": 1, "mali": 1, "bahamas": 1, "sudan": 1, "benin": 1, "latvia": 1, "tunisia": 1,
        "guyana": 1, "gabon": 1, "cyprus": 1, "syria": 1, "georgia": 1, "libya": 1, "bermuda": 1, "austria": 1, "zambia": 1, "fiji": 1, "macedonia": 1, "brunei": 1, "norway": 1, "caymanislands": 1, "kazakhstan": 1, "newzealand": 1
    },

    "region":{
        "LGEAG": "LA", "LGECZ": "EU", "LGEFS": "EU", "LGEDG": "EU", "LGEHS": "EU", "LGEMK": "EU", "LGEIS": "EU", "LGESC": "EU", "LGEEH": "EU", "LGEBN": "EU", "LGEWR": "EU", "LGEPL": "EU", "LGEMA": "EU", "LGEPT": "EU", "LGERO": "EU",
        "LGEES": "EU", "LGENO": "EU", "LGESW": "EU", "LGEUK": "EU", "LGEAK": "OT", "LGERM": "OT", "LGERI": "OT", "LGERA": "OT", "LGEUR": "OT", "LGELV": "OT", "LGEAS": "OT", "LGEEG": "OT", "LGELF": "OT", "LGESK": "OT", "LGEMC": "OT",
        "LGESA": "OT", "LGETU": "OT", "LGEOT": "OT", "LGEDF": "OT", "LGEGF": "OT", "LGEME": "OT", "LGEAF": "OT", "LGEAO": "OT", "LGENI": "OT", "LGETK": "OT", "LGEAT": "OT", "LGESJ": "OT", "LGEEF": "OT", "LGEYK": "OT", "LGEIR": "OT",
        "LGEEB": "OT", "LGELA": "OT", "LGEBT": "OT", "LGEAP": "AP", "LGEQA": "AP", "LGETL": "AP", "LGECH": "AP", "LGEYT": "AP", "LGETR": "AP", "LGETA": "AP", "LGESY": "AP", "LGESH": "AP", "LGEQH": "AP", "LGEQD": "AP", "LGEPN": "AP",
        "LGENE": "AP", "LGEKS": "AP", "LGEHZ": "AP", "LGEHN": "AP", "LGEHK": "AP", "LGEIL": "AP", "LGEPH": "AP", "LGEVH": "AP", "LGEKR": "AP", "LGESL": "AP", "LGEIN": "AP", "LGETH": "AP", "LGEML": "AP", "LGETT": "AP", "LGEJP": "AP",
        "LGECI": "NA", "LGERS": "NA", "LGEMX": "NA", "LGEMS": "NA", "LGEMM": "NA", "LGEMR": "NA", "LGEUS": "NA", "LGEMU": "NA", "LGEAI": "NA", "LGEBR": "LA", "LGECL": "LA", "LGEVZ": "LA", "LGECB": "LA", "LGEPS": "LA", "LGEPR": "LA",
        "LGESP": "LA", "LGEAR": "LA"
    },

    "inquiry_type":{"quotation":"QP", "purchase":"QP", "sales":"SA", "dump_key":"OT"},

    "customer_position":{"none":"NO", "manager":"MA", "founder":"FD", "director":"DR", "entry":"EN", "analyst":"AN", "partner":"PA", "level":"LV", "execut":"EX", "c-level":"CL", "traine":"TR", "presid":"PR", "vice":"VI", "intern":"IN"},

    "values":{
        "customer_country":"dump_value",
        "business_unit":["ID", "AS", "IT", "ETC"],
        "customer_job":["OT", "EN", "AD", "ED", "SA", "PA", "OP", "IF", "AT", "BS", "MA", "OT"],
        "inquiry_type":["QP", "SA", "OT"],
        "product_category":["OT", "SG", "VR", "MS", "SS", "TV", "OT"],
        "customer_position":["NO", "OT", "MA", "FD", "DR", "AN", "PA", "EN", "OT"],
        "response_corporate":["AP", "LA", "NA", "EU", "OT"]
    },
    "response_corporate":"dump_value",
    "business_area":"dump_value",
    "enterprise":"dump_value",
    "business_unit":"dump_value",

    "com_reg_ver_win_rate":"dumpy_value",
    "customer_type":{"endcustomer":"EC", "specifier/influencer":"SI", "channelpartner":"CP", "dump_key":"OT"},
    "expected_timeline":{"lessthan3months":"L3", "3months~6months":"36", "morethanayear":"MY", "9months~1year":"91", "6months~9months":"69", "dump_key":"OT"}
}

### 2.2. 삭제 컬럼
> Train 데이터셋 기준 결측치가 80% 이상인 컬럼 삭제 <br>
> customer_country.1 컬럼은 customer_country 컬럼과 대부분 일치하여 삭제

In [146]:
# delete cols
del_cols=['business_subarea', 'product_subcategory', 'product_modelname', 
          'customer_country.1']

# preserve
# preserve=pd.DataFrame()
# preserve['com_reg_ver_win_rate']=total_data['com_reg_ver_win_rate']

train_process=train.drop(del_cols,axis=1)
test_process=test.drop(del_cols,axis=1)

### 2.3. strategic_ver
> id_strategic_ver, it_strategic_ver, idit_strategic_ver 컬럼의 경우에는 존재하지 않으면 가중치가 존재하지 않음 <br>
> 따라서 strategic_ver의 단일 컬럼으로 대체 <br><br>

In [147]:
# id_strategic_ver it_strategic_ver idit_strategic_ver
ver=['id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver']
train_process['strategic_ver']=np.where(train_process['idit_strategic_ver']>0,1,0)
test_process['strategic_ver']=np.where(test_process['idit_strategic_ver']>0,1,0)
train_process=train_process.drop(ver,axis=1)
test_process=test_process.drop(ver,axis=1)

### 2.4. 국적 관련 데이터 처리
> 코드 별 주석 내용 참조<br>

In [148]:
# country columns

# 허용 변수인 permitted를 Train 데이터셋 기준으로 재정의
def response_corporate_encoding(train_data):
    # 자사법인명 컬럼 내용을 허용 변수로 사용 
    permit={}
    
    for train_label in train_data.value_counts().index:
        permit[train_label]=1
    permit['OT']=1

    return permit

def country_encoding(permitted):
    # 고객 국적 데이터 중 turkey 등을 위한 예외사항 적용 함수
    permit={}
    for per in permitted.keys():
        permit[per]=per

    permit['OT']='OT'
    permit['dump_key']='dump_value'
    permit['türkiye']='turkey'
    
    return permit

# 허용 변수 적용
def preprocess_region(x,permitted):
    if type(x)==type(''):
        if permitted.get(x):
            return permitted[x]
        return 'OT'
    return np.nan

def preprocess_response_corporate(x,permitted):
    if type(x)==type(''):
        if permitted.get(x):
            return x
        return 'OT'
    return np.nan

def preprocess_customer_country(x,permitted):
    if type(x)==type(''):
        x=x.lower().replace(' ','').replace('/',' ')
        for word in x.split(' '):
            if permitted.get(word):
                return word
        return 'OT'
    return np.nan

# region
train_process['region']=train_process['response_corporate'].apply(lambda x:preprocess_region(x,permitted=permitted['region']))
test_process['region']=test_process['response_corporate'].apply(lambda x:preprocess_region(x,permitted=permitted['region']))

# response_corporate
permitted['response_corporate']=response_corporate_encoding(train['response_corporate'])
train_process['response_corporate']=train_process['response_corporate'].apply(lambda x:preprocess_response_corporate(x,permitted=permitted['response_corporate']))
test_process['response_corporate']=test_process['response_corporate'].apply(lambda x:preprocess_response_corporate(x,permitted=permitted['response_corporate']))

# customer_country   
permitted['customer_country']=country_encoding(permitted['customer_country'])
train_process['customer_country']=train_process['customer_country'].apply(lambda x:preprocess_customer_country(x,permitted=permitted['customer_country']))
test_process['customer_country']=test_process['customer_country'].apply(lambda x:preprocess_customer_country(x,permitted=permitted['customer_country']))

### 2.5. business_unit
> Train 데이터셋 기준으로 CM 변수의 빈도 너무 적어 다음으로 빈도가 적은 Solution 변수와 병합<br>

In [149]:
# business_unit
train_process['business_unit']=train_process['business_unit'].replace('Solution','ETC')
train_process['business_unit']=train_process['business_unit'].replace('CM','ETC')

test_process['business_unit']=test_process['business_unit'].replace('Solution','ETC')
test_process['business_unit']=test_process['business_unit'].replace('CM','ETC')

### 2.6. customer_type
> 대문자, 빈출 특수문자 처리 후 허용 변수 내로 범주화<br>

In [150]:
# customer_type
def preprocess_customer_type(x,permitted):
    if type(x)==type(''):
        x=x.lower().replace('-','').replace(' ','')
        if permitted.get(x):
            return permitted[x]
        else:
            return 'OT'
    return x
    
train_process['customer_type']=train_process['customer_type'].apply(lambda x:preprocess_customer_type(x,permitted=permitted['customer_type']))
test_process['customer_type']=test_process['customer_type'].apply(lambda x:preprocess_customer_type(x,permitted=permitted['customer_type']))

### 2.7. grant_weight
> vertical 관련 가중치 컬럼의 경우에도 strategic_ver 컬럼과 마찬가지로 이유로 통합하여 grant_weight 컬럼 신설<br>

In [151]:
# ver_cus, ver_pro
grant=['ver_cus', 'ver_pro']
train_process['grant_weight']=np.where(train_process['ver_cus']>0,1,0)
train_process['grant_weight']=np.where(train_process['ver_pro']>0,1,train_process['grant_weight'])
train_process=train_process.drop(grant,axis=1)

test_process['grant_weight']=np.where(test_process['ver_cus']>0,1,0)
test_process['grant_weight']=np.where(test_process['ver_pro']>0,1,test_process['grant_weight'])
test_process=test_process.drop(grant,axis=1)

### 2.8. expected_timeline
> 대문자, 빈출 특수문자 처리 후 허용 변수 내로 범주화<br>

In [152]:
# expected_timeline
def preprocess_expected_timeline(x,permitted):
    if type(x)==type(''):
        x=x.lower().replace(' ','').replace('_','')
        if permitted.get(x):
            return permitted[x]
        return 'OT'
    return x

train_process['expected_timeline']=train_process['expected_timeline'].apply(lambda x:preprocess_expected_timeline(x,permitted=permitted['expected_timeline']))
test_process['expected_timeline']=test_process['expected_timeline'].apply(lambda x:preprocess_expected_timeline(x,permitted=permitted['expected_timeline']))

### 2.9. 수치형 칼럼 처리
> 수치형 칼럼 중 bant_submit를 제외한 컬럼들을 RobustScaler를 사용하여 스케일링<br>
> 이상치에 덜 민감한 robust scale 채택

In [153]:
from sklearn.preprocessing import RobustScaler

numerical=['lead_desc_length','historical_existing_cnt','com_reg_ver_win_rate','ver_win_rate_x','ver_win_ratio_per_bu']
scaler=RobustScaler()
scaler.fit(train_process[numerical])
train_process[numerical]=scaler.transform(train_process[numerical])
test_process[numerical]=scaler.transform(test_process[numerical])

### 2.10. inquiry_type
> 대문자, 빈출 특수문자 처리 후 허용 변수 내로 범주화<br>

In [154]:
# inquiry_type
def preprocess_inquiry_type(x,permitted):
    if type(x)==type(''):
        x=x.lower().replace('_',' ')
        for word in x.split(' '):
            if permitted.get(word):
                return permitted[word]
        return 'OT'
    return np.nan

train_process['inquiry_type']=train_process['inquiry_type'].apply(lambda x:preprocess_inquiry_type(x,permitted=permitted['inquiry_type']))
test_process['inquiry_type']=test_process['inquiry_type'].apply(lambda x:preprocess_inquiry_type(x,permitted=permitted['inquiry_type']))

### 2.11. customer_job, product_category
> 표제어 기준으로 데이터 범주화
> 단어 추출을 진행한 후에 porter stem 기준으로 빈출 단어로 범주화<br>
> Porter stemmer : https://tartarus.org/martin/PorterStemmer/

In [155]:
# customer_job
def preprocess_customer_job(x,permitted):
    # 빈출 단어가 있으면 해당 단어로 없으면 'OT' -> others로 범주화
    if type(x)==type(''):
        porter=PorterStemmer()
        tokens=word_tokenize(x)
        stems=[porter.stem(token) for token in tokens]
        for stem in stems:
            if permitted.get(stem):
                return permitted[stem]
        return 'OT'
    return np.nan

train_process['customer_job']=train_process['customer_job'].apply(lambda x:preprocess_customer_job(x,permitted=permitted['customer_job']))
test_process['customer_job']=test_process['customer_job'].apply(lambda x:preprocess_customer_job(x,permitted=permitted['customer_job']))

In [156]:
# product_category
def preprocess_product_category(x,permitted):
    # 빈출 단어가 있으면 해당 단어로 없으면 'OT' -> others로 범주화
    # dump_key에 담긴 우선순위 기준으로 하나의 범주만 채택  ex) TV, SG가 모두 검출되면 우선순위가 높은 SG로 판단
    if type(x)==type(''):
        porter=PorterStemmer()
        tokens=word_tokenize(x)
        stems=[porter.stem(token) for token in tokens]
        
        # dump_key에 담긴 우선순위 사용
        prefer={}
        for pf in permitted['dump_key']:
            prefer[pf]=0
        
        for stem in stems:
            if permitted.get(stem):
                prefer[permitted[stem]]=1

        for pf in permitted['dump_key']:
            if prefer[pf]>0:
                return pf
        return 'OT'
    return np.nan

train_process['product_category']=train_process['product_category'].apply(lambda x:preprocess_product_category(x,permitted=permitted['product_category']))
test_process['product_category']=test_process['product_category'].apply(lambda x:preprocess_product_category(x,permitted=permitted['product_category']))

### 2.12. customer_position
> 대문자, 빈출 특수문자 처리 후 허용 변수 내로 범주화<br>

In [157]:
# customer_poisition
def preprocess_customer_position(x,permitted):
    if type(x)==type(''):
        x=x.lower().replace('-',' ').replace('/',' ')
        for word in x.split(' '):
            if permitted.get(word):
                return permitted[word]
        return 'OT'
    return np.nan

train_process['customer_position']=train_process['customer_position'].apply(lambda x:preprocess_customer_position(x,permitted=permitted['customer_position']))
test_process['customer_position']=test_process['customer_position'].apply(lambda x:preprocess_customer_position(x,permitted=permitted['customer_position']))

## 3. 결측치 대체
> 범주형 컬럼은 order labeling 진행<br>
> MICE 알고리즘으로 결측치 대체 진행, 회귀 모델은 랜덤포레스트 모델 사용<br>
> 결정트리 기반 모델이기 때문에 order labeling를 채택하여 진행함<br><br>
> Data leakage 방지를 위해 MICE에 사용되는 회귀 모델은 Train 데이터셋으로만 가중치 업데이트 진행<br>
> 마찬가지로 Data leakage 방지를 위해 encoder 또한 Train 데이터셋으로만 fitting 진행<br>
> Test 데이터셋에 경우에는 Train 데이터셋으로 학습시킨 모델로 예측만 진행

In [158]:
# encoding columns
# 라벨링 진행을 위해 범주형 칼럼 추출 및 확인
origin_data=train_process.drop('com_reg_ver_win_rate',axis=1)
origin_columns=origin_data.columns.to_list()
object_columns=origin_data.columns.to_list()
object_columns.remove('bant_submit')
object_columns.remove('historical_existing_cnt')
object_columns.remove('lead_desc_length')
object_columns.remove('strategic_ver')
object_columns.remove('grant_weight')
object_columns.remove('customer_idx')
object_columns.remove('lead_owner')
object_columns.remove('ver_win_rate_x')
object_columns.remove('ver_win_ratio_per_bu')
for col in object_columns:
    permitted['values'][col]=train_process[col].value_counts().index
permitted['values']

{'customer_country': Index(['india', 'brazil', 'unitedstates', 'mexico', 'OT', 'philippines',
        'colombia', 'u.a.e', 'unitedkingdom', 'saudiarabia',
        ...
        'fiji', 'caymanislands', 'laos', 'benin', 'sierraleone', 'congo',
        'unitedarabemirates', 'gambia', 'mali', 'ivorycoast'],
       dtype='object', length=134),
 'business_unit': Index(['ID', 'AS', 'IT', 'ETC'], dtype='object'),
 'customer_job': Index(['OT', 'EN', 'AD', 'ED', 'SA', 'PA', 'OP', 'IF', 'AT', 'BS', 'MA', 'CS',
        'ME', 'DV', 'TC'],
       dtype='object'),
 'inquiry_type': Index(['QP', 'SA', 'OT'], dtype='object'),
 'product_category': Index(['OT', 'SG', 'IT', 'VR', 'MS', 'SS', 'SE', 'TV', 'VI', 'LD'], dtype='object'),
 'customer_position': Index(['NO', 'OT', 'MA', 'FD', 'DR', 'AN', 'PA', 'EN', 'LV', 'VI', 'IN'], dtype='object'),
 'response_corporate': Index(['LGEIL', 'LGESP', 'LGEUS', 'LGEMS', 'LGEPH', 'LGEGF', 'LGECB', 'LGEUK',
        'LGESJ', 'LGECL', 'LGEPS', 'LGEIS', 'LGEPR', 'LGEDG', 'L

In [159]:
# encoder
class Encoder():
    '''
    order labeling를 위한 클래스
    범주형의 변수들을 숫자로 변환
    결측치는 유지
    '''
    def __init__(self):
        self.classes=[]

    def fit(self,data):
        for value in data.value_counts().index:
            self.classes.append(value)

    def transform(self,data):
        result=data.copy(deep=True)
        for i,value in enumerate(self.classes):
            result=result.replace(value,i)
        return result
    
    def inverse_transform(self,data):
        result=data.copy(deep=True)
        for i in range(0,len(self.classes)):
            result=result.replace(i,self.classes[i])
        return result

In [160]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

# 랜덤포레스트로 impute 모델 준비, 초기에는 최빈값으로 예측
imputer=IterativeImputer(estimator=RandomForestRegressor(random_state=SEED),initial_strategy='most_frequent',max_iter=10,random_state=SEED,skip_complete=False,verbose=1)

# encoder
# Train 데이터셋 기준으로만 order label 준비
encoders={}
for object_column in object_columns:
    encoders[object_column]=Encoder()
    encoders[object_column].fit(train_process[object_column])

# 라벨링 관련 encoding, decoding 함수
def order_encoding(target_data,object_columns,encoders):
    result=pd.DataFrame()
    for col in target_data.columns:
        if col in object_columns:
            result[col]=encoders[col].transform(target_data[col])
        else:
            result[col]=target_data[col]

    return result
def order_decoding(target_data,object_columns,encoders):
    result=pd.DataFrame()
    for col in target_data.columns:
        if col in object_columns:
            result[col]=target_data[col].apply(lambda x:round(x)).astype(int)
            result[col]=encoders[col].inverse_transform(result[col])
        else:
            result[col]=target_data[col]
    return result

# train data
train_dummy=order_encoding(train_process,object_columns,encoders)
imputer.fit(train_dummy)    # 회귀 모델의 fit은 오직 Train 데이터셋으로만 진행
train_dummy_imputed=pd.DataFrame(data=imputer.transform(train_dummy),columns=train_process.columns)
train_imputed=order_decoding(train_dummy_imputed,object_columns,encoders)

# test data
test_dummy=order_encoding(test_process,object_columns,encoders)
test_dummy_imputed=pd.DataFrame(data=imputer.transform(test_dummy),columns=test_process.columns)
test_imputed=order_decoding(test_dummy_imputed,object_columns,encoders)


[IterativeImputer] Completing matrix with shape (59299, 22)
[IterativeImputer] Change: 80.05833333333334, scaled tolerance: 47.466 
[IterativeImputer] Change: 26.809685928239805, scaled tolerance: 47.466 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (59299, 22)
[IterativeImputer] Completing matrix with shape (5271, 22)


## 4. 데이터 폴드 및 타입 처리
> 타겟값의 분포를 고려하여 목표가 희소값이기 때문에 데이터셋을 비율을 맞추어 폴드하여 진행<br>
> 희소값인 positive rows은 전부 사용하여 1:1 비율로 negative rows은 랜덤 샘플링하여 병합<br>
> positive:negative 비율을 고려하여 총 11개의 데이터셋으로 분할

In [161]:
# data kfold
from sklearn.model_selection import KFold

train_datas=[]
train_imputed['is_converted']=targets
train_data_false=train_imputed[train_imputed['is_converted']==0]    # negative 데이터셋
train_data_true=train_imputed[train_imputed['is_converted']==1]     # positive 데이터셋

# kfold
K=11
dkf=KFold(n_splits=K,shuffle=True,random_state=SEED)
for i,(_,index) in enumerate(dkf.split(train_data_false)):
    print(f'-{i+1} fold data-')
    data_false=train_data_false.iloc[index]     # 랜덤 추출된 negative 데이터
    
    full_data=pd.concat([data_false,train_data_true],ignore_index=True)     # 데이터 병합
    full_data=full_data.sample(frac=1,random_state=SEED)                    # 병합된 데이터 셔플
    X_data=full_data.drop('is_converted',axis=1)
    y_data=full_data['is_converted']

    print(f'X data shape: {X_data.shape}')
    print(f'y data shape: {y_data.shape}')
    train_datas.append((X_data,y_data))

test_data=test_imputed
print(f'test data shape: {test_data.shape}')

-1 fold data-
X data shape: (9800, 22)
y data shape: (9800,)
-2 fold data-
X data shape: (9800, 22)
y data shape: (9800,)
-3 fold data-
X data shape: (9800, 22)
y data shape: (9800,)
-4 fold data-
X data shape: (9800, 22)
y data shape: (9800,)
-5 fold data-
X data shape: (9800, 22)
y data shape: (9800,)
-6 fold data-
X data shape: (9800, 22)
y data shape: (9800,)
-7 fold data-
X data shape: (9800, 22)
y data shape: (9800,)
-8 fold data-
X data shape: (9800, 22)
y data shape: (9800,)
-9 fold data-
X data shape: (9800, 22)
y data shape: (9800,)
-10 fold data-
X data shape: (9800, 22)
y data shape: (9800,)
-11 fold data-
X data shape: (9799, 22)
y data shape: (9799,)
test data shape: (5271, 22)


In [162]:
# process data type
# 전체 데이터셋의 타입 변경
for (train_data,target) in train_datas:
    train_data['customer_idx']=train_data['customer_idx'].astype(str)
    train_data['lead_owner']=train_data['lead_owner'].astype(str)
    train_data['strategic_ver']=train_data['strategic_ver'].astype(int)
    train_data['grant_weight']=train_data['grant_weight'].astype(int)
    target=target.apply(lambda x:1 if x else 0)

test_data['customer_idx']=test_data['customer_idx'].astype(str)
test_data['lead_owner']=test_data['lead_owner'].astype(str)
test_data['strategic_ver']=test_data['strategic_ver'].astype(int)
test_data['grant_weight']=test_data['grant_weight'].astype(int)

## 5. 모델링
> 분할된 데이터셋을 활용하여 여러 예측 모델의 학습을 진행<br>
> 범주형 데이터의 비중이 높아 약분류기는 범주형 데이터에 좋은 성능을 보여주는 Catboost 모델 채택<br>
> 각각의 예측 모델에서 예측한 값을 바탕으로 앙상블하여 최종 판단<br><br>
> 분할된 데이터셋 마다 k-fold 방식을 사용해 k개의 예측 모델을 생성<br>
> 총 (분할된 데이터셋 수)x(k-fold 수)=11x5=55개의 예측 모델 사용<br><br>
> Robust한 모델을 위해 positive인 확률을 예측하는 회귀 모델을 사용하고, label smothing을 진행<br>
> label smothing 진행 시에 랜덤으로 발생시킨 noise 값으로 각 확률 0,1에서 증감하여 사용

In [163]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics         import mean_squared_error

In [164]:
# 모델 성능 테스트
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))
    return F1

In [165]:
class KMODEL:
    '''
    분할된 데이터셋을 기준으로 여러개의 예측 모델을 사용하는 클래스
    '''
    def __init__(self,dataset_K,train_K=5,random_state=SEED):
        self.k_data=dataset_K                               # 분할된 데이터셋 수
        self.k_fold=train_K                                 # 사용할 k-fold의 k
        self.models=[[] for i in range(0,self.k_data)]      # 예측 모델
        self.scores=[[] for i in range(0,self.k_data)]      # 각 예측 모델의 f1 score
        self.thresholds=[[] for i in range(0,self.k_data)]  # 각 예측 모델의 threshold
        self.cv_scores=[]                                   # 분할 데이터 기준 평균 f1 score
        self.final_threshold=0.45                           # 예측 시에 최종적으로 사용할 threshold
        self.seed=random_state                              # random seed

    def modeling_kfold(self,iters,n_estimators,max_depth,learning_rate,cat_features,train_data,targets_data,noise,core):
        '''
        iter : 분할된 데이터셋 id
        n_estimator,max_depth,learning_rate,cat_features : 회귀 모델에 사용할 하이퍼 파라미터
        train_data, targets_data : 분할한 일부 학습 데이터
        noise : label smothing 위해 발생시킨 노이즈
        core : CPU or GPU
        '''
        # k-fold
        kf=StratifiedKFold(n_splits=self.k_fold,shuffle=True,random_state=self.seed)

        for i,(train_index,val_index) in enumerate(kf.split(train_data,targets_data)):
            print(f'-[{iters+1}-{i+1}] fold-')
            # noise
            targets_noised=(targets_data+noise).apply(lambda x:self.value_scale(x))     # 노이즈 적용

            # 학습, 검증 데이터 분할
            X_train,X_val=train_data.iloc[train_index],train_data.iloc[val_index]
            y_train,y_val=targets_noised.iloc[train_index],targets_noised.iloc[val_index]

            # regressor로 모델 학습
            regressor=CatBoostRegressor(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, eval_metric='RMSE',random_state=self.seed, bootstrap_type ='Bernoulli',task_type=core)

            model=regressor.fit(X_train, y_train, eval_set=(X_val,y_val),verbose=100, early_stopping_rounds=100,cat_features=cat_features,use_best_model=True)
            pred=model.predict(X_val)

            # 분할된 k-fold 모델 기준으로 최적의 threshold 찾기 (검증 데이터셋 활용)
            coordinates = np.linspace(0, 1, 100)
            y_val=y_val>0.5
            best_score=0
            best_coordinate=0
            for coordinate in coordinates:
                pred_value=pred>coordinate
                score=f1_score(y_val,pred_value)
                if best_score<score:
                    best_score=score
                    best_coordinate=coordinate
            
            # 검증 데이터셋 기준으로 모델 생성
            pred=(pred>best_coordinate)
            self.scores[iters].append(get_clf_eval(y_val,pred))
            self.thresholds[iters].append(best_coordinate)
            self.models[iters].append(model)
        
        self.cv_scores.append(np.mean(self.scores[iters]))      # 분할된 데이터셋 기준 평균 f1 score
        print(f'[{iters+1}] F1 scores mean: {self.cv_scores[iters]}')

    def modeling_kdata(self,n_estimators,max_depth,learning_rate,cat_features,train_datas,core='CPU'):
        '''
        n_estimator,max_depth,learning_rate,cat_features : 회귀 모델에 사용할 하이퍼 파라미터
        train_datas: 분할한 전체 학습 데이터
        core : CPU or GPU
        '''

        # 랜덤 노이즈 발생 노이즈의 범위는 -0.1~0.1, 소수점 5자리까지
        noise_size=0
        for _,target in train_datas:
            noise_size+=target.shape[0]
        noises=[round(random.uniform(-0.1,0.1),5) for i in range(noise_size)]

        # 노이즈를 적용하여 분할한 데이터셋 별로 학습 진행
        checksum=0
        for iter,(train_data,target) in enumerate(train_datas):
            noise=noises[checksum:checksum+target.shape[0]]
            self.modeling_kfold(iter,n_estimators,max_depth,learning_rate,cat_features,train_data,target,noise,core=core)
            checksum+=target.shape[0]
            
        self.final_threshold=np.mean(self.thresholds)       # 최종 threshold는 약분류기의 threshold의 평균 채택
        print(f'-----Total F1 scores mean: {np.mean(self.cv_scores)}-----')
        


    def predict(self,test_data):
        # 두 버전을 준비하여 예측 진행 -> version2 채택 중
        test_pred=pd.Series([0 for x in range(len(test_data))], index=test_data.index)
        
        # # version1
        # # 각 약분류기의 threshold를 사용하여 이진분류를 진행하고 과반수 이상의 약분류기가 positive이면 positive로 예측
        # for models,thresholds in zip(self.models,self.thresholds):
        #     for model,threshold in zip(models,thresholds):
        #         pred=model.predict(test_data)
        #         test_pred+=pred>threshold
        # test_pred=test_pred/((self.k_data)*(self.k_fold))
        # test_pred=test_pred.apply(lambda x:1 if x>0.5 else 0)
        
        # version2
        # 각 약분류기의 예측 값(확률)의 평균을 구하고 최종 threshold 값을 기준으로 positive 판단
        for models in self.models:
            for model in models:
                pred=model.predict(test_data)
                test_pred+=pred
        test_pred=test_pred/((self.k_data)*(self.k_fold))
        test_pred=test_pred>self.final_threshold
        
        return test_pred
    
    def value_scale(self,x):
        # 모델이 확률을 예측할 것이기에 0~1 범위를 가지게 하기위한 scaler
        if x<0:
            return 0
        elif x>1:
            return 1
        
        return x

In [166]:
# 범주형 컬럼
cat_features=train_datas[0][0].columns.to_list()
cat_features.remove('bant_submit')
cat_features.remove('lead_desc_length')
cat_features.remove('historical_existing_cnt')
cat_features.remove('com_reg_ver_win_rate')
cat_features.remove('ver_win_rate_x')
cat_features.remove('ver_win_ratio_per_bu')
cat_features

['customer_country',
 'business_unit',
 'customer_idx',
 'customer_type',
 'enterprise',
 'customer_job',
 'inquiry_type',
 'product_category',
 'customer_position',
 'response_corporate',
 'expected_timeline',
 'business_area',
 'lead_owner',
 'strategic_ver',
 'region',
 'grant_weight']

In [167]:
# 모델 학습 진행
kmodel=KMODEL(dataset_K=11)
kmodel.modeling_kdata(n_estimators=1000,max_depth=8,learning_rate=0.05,cat_features=cat_features,train_datas=train_datas)

-[1-1] fold-
0:	learn: 0.4640228	test: 0.4644977	best: 0.4644977 (0)	total: 45.2ms	remaining: 45.1s
100:	learn: 0.2292240	test: 0.2307925	best: 0.2307925 (100)	total: 4.84s	remaining: 43.1s
200:	learn: 0.2128958	test: 0.2266238	best: 0.2266197 (199)	total: 9.38s	remaining: 37.3s
300:	learn: 0.1986051	test: 0.2233828	best: 0.2233828 (300)	total: 13.5s	remaining: 31.3s
400:	learn: 0.1861259	test: 0.2213295	best: 0.2212836 (395)	total: 17.6s	remaining: 26.3s
500:	learn: 0.1754618	test: 0.2204764	best: 0.2204733 (499)	total: 21.7s	remaining: 21.7s
600:	learn: 0.1656358	test: 0.2195620	best: 0.2195620 (600)	total: 25.8s	remaining: 17.1s
700:	learn: 0.1560251	test: 0.2185678	best: 0.2185678 (700)	total: 30s	remaining: 12.8s
800:	learn: 0.1482133	test: 0.2178675	best: 0.2178329 (795)	total: 34.1s	remaining: 8.48s
900:	learn: 0.1413401	test: 0.2175148	best: 0.2174643 (887)	total: 38.3s	remaining: 4.21s
999:	learn: 0.1346592	test: 0.2169217	best: 0.2168873 (989)	total: 42.6s	remaining: 0us

bes

In [168]:
# 예측 진행
pred=kmodel.predict(test_data)
pred.value_counts()

False    3639
True     1632
dtype: int64

In [169]:
# 제출 파일 생성
pred=pred.apply(lambda x:1 if x else 0)
submission=pd.read_csv('submission.csv')
submission['is_converted']=pred
submission.to_csv('submission.csv',index=False)
submission.head()

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,1
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,NaN,...,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,1
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,0
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,0
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,1


**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**